In [2]:
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

delay = 3
browser = Chrome("./chromedriver")
browser.implicitly_wait(delay)
browser.get("https://www.youtube.com/channel/UCN8CPzwkYiDVLZlgD4JQgJQ/videos") # 박막례할머니 동영상 페이지로 들어감
browser.maximize_window()
body = browser.find_element_by_tag_name('body') # body태그 가져오기

num_of_pagedowns = 22
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1 #video페이지의 스크롤을 충분히 밑으로 내려놓고

In [3]:
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')
video_list2 = html.select('div #dismissable')

youtube_url = 'http://www.youtube.com'

park_url = []

print("video_list2의 길이:",len(video_list2))
for i in range(len(video_list2)):
    url = youtube_url + video_list2[i].find('a',{'id': 'thumbnail'})['href']
    park_url.append(url) # 모든 영상의 url을 youtube_url에 붙여서 완전한 url로 만들고 park_url이라는 리스트에 넣어줌

video_list2의 길이: 272


In [ ]:
# 현재로서는 썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요, 만 크롤링 하면 될듯!

# park = pd.DataFrame({'thumbnail': [],
#                      'title' : [],
#                      'category': [],
#                      'view_count': [],
#                      'likes_count': [],
#                      'dislikes_count':[],
#                      'video_length':[],
#                      'upload_date': [],
#                      }) #총 8개의 정보

for i in range(0, 20):
    #여기까지는 video페이지에서 수집할 수 있는 썸네일, 제목을 가져온다.
    thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
    title = video_list2[i].find('a',{'id':'video-title'}).text # 2 
    
    # 여기서부터 영상 클릭해서 들어간 페이지
    click_url = park_url[i]
    browser.get(click_url)
    time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
    # body = browser.find_element_by_tag_name('body')
    browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
    time.sleep(3)
    html0 = browser.page_source
    html = BeautifulSoup(html0, 'html.parser')
   
    # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
    category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
    view_count = html.find('span', {'class': 'view-count'}).text # 4
    
    # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
    # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
    likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
    'aria-label':re.compile('좋아요')}).text+'개'
    dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
    'aria-label':re.compile('싫어요')}).text+'개'
    video_length = html.find('span', {'class': 'ytp-time-duration'}).text # 7
    upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8
    
    print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "영상길이", video_length, "게시일", upload_date)
#     # 크롤링한 정보 저장  
#     insert_data = pd.DataFrame({
#                         'thumbnail': [thumbnail],
#                         'title' : [title],
#                         'category': [category],
#                         'view_count': [view_count],
#                         'likes_count': [likes_count],
#                         'dislikes_count':[dislikes_count],
#                         'video_length':[video_length],
#                         'upload_date': [upload_date],
#                         }) 

#     park = park.append(insert_data)
# park.index = range(len(park))

0 . 썸네일 https://i.ytimg.com/vi/1LW33ylk8wM/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAKMobJzYv2PTYQsoZ6vJKWonw7lg 제목: 박막례 간장게장 만들기 [박막례 할머니] 카테고리 코미디 조회수 조회수 611,297회 좋아요수 1.9만개 싫어요수 223개 영상길이 0:30 게시일 2020. 1. 5.
1 . 썸네일 https://i.ytimg.com/vi/BnmL_KbnStE/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAtKUvlAfw_Tr8TbtOVezuPGN3qyg 제목: 핫팩 사세요~ 호~ HOT핫HOT핫팩이 왔어요~❤️ [박막례 할머니] 카테고리 코미디 조회수 조회수 126,251회 좋아요수 7.4천개 싫어요수 31개 영상길이 0:15 게시일 2020. 1. 3.
2 . 썸네일 https://i.ytimg.com/vi/nbQQauKcHmk/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDVsZI6akM_iSkowKHTchRmfaME5w 제목: 그 연예대상 시상식 브이로그!!!  [박막례 할머니] 카테고리 코미디 조회수 조회수 538,250회 좋아요수 2.4만개 싫어요수 141개 영상길이 0:30 게시일 2019. 12. 29.
3 . 썸네일 https://i.ytimg.com/vi/NrTC1wqiGVQ/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAyCTp96M5hXLjXPM_UbJUCUw2xpQ 제목: 크리스마스 홈파티 인 파리 [박막례 할머니] 카테고리 코미디 조회수 조회수 246,574회 좋아요수 1만개 싫어요수 73개 영상길이 7:37 게시일 2019. 12. 24.